In [50]:
import os
import numpy as np

# Provide the path to the overall experiment folder and the path where you would like the merged file to be saved
folder_path = 'C:/Users/aewor/Documents/Python/EmanuelLabRotation/2024-01-05_ALC4_day3'
output_path = 'C:/Users/aewor/Documents/Python/EmanuelLabRotation/merged_test'

In [58]:
#Empty list will contain filepaths of recordings that are to be concatenated
files_to_concat = []

#OpenEphys files will have subdirectories for every set of experimental recordings
subdirectories = [item for item in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, item))]
subdirectories = sorted(subdirectories, key=lambda x: int(''.join(filter(str.isdigit, x))))

for folder in subdirectories:
    node_path = os.path.join(folder_path, folder, 'Record Node 101').replace('\\', '/')
    if os.path.exists(node_path) and os.path.isdir(node_path):
        #nodes in subdirectories may have more than one experiment file in them.
        experiments = [item for item in os.listdir(node_path) if os.path.isdir(os.path.join(node_path, item))]
        experiments = sorted(experiments, key=lambda x: int(''.join(filter(str.isdigit, x))))

        if len(experiments) >= 1:
            #Each of these experiment files can contain multiple recordings.
            for experiment in experiments:
                exp_path = os.path.join(node_path,experiment).replace('\\', '/')
                recordings = [item for item in os.listdir(exp_path) if os.path.isdir(os.path.join(exp_path, item))]
                recordings = sorted(recordings, key=lambda x: int(''.join(filter(str.isdigit, x))))

            if len(recordings) >= 1:
                for recording in recordings:
                    recording_path = os.path.join(exp_path, recording,'continuous','Neuropix-PXI-100.ProbeA-AP','continuous.dat').replace('\\', '/')
                    files_to_concat.append(recording_path)
            else:
                print(f"No recording directories found in {exp_path}.")
        else:
            print(f"No experiment directories found in {node_path}.")        
    else:
        print(f"No 'Record Node 101' directory found in {os.path.join(folder_path, folder)}.")

print(len(files_to_concat))

175


In [59]:
def openephys_concat (recording_filepaths, save_path):
    raw_combined = open(os.path.join(save_path, 'combined_trial.DAT').replace('\\', '/'), 'wb')

    for recording in recording_filepaths:
        to_append = np.memmap(recording, mode='r')
        to_append.tofile(raw_combined)
            
    raw_combined.close()
    print('Concatenation complete.')

openephys_concat(files_to_concat, output_path)

Concatenation complete.


In [57]:
## input list of continuous.DAT filepaths to be concatenated. Will save a concatenated version in specified save_path
# def mne_concat (filepaths, save_path):
#     raw_combined = None

#     for path in filepaths:
#         raw = mne.io.read_raw_openephys(path, preload=True)
#         if raw_combined is None:
#             raw_combined = raw
#         else:
#             raw_combined.append(raw)
            
#     raw_combined.save('{}/combined_dataset.fif'.format(output_path))